In [1]:
from acdesign.atmosphere import Atmosphere
from acdesign.performance.operating_point import OperatingPoint
from acdesign.airfoils.polar import UIUCPolar
from acdesign.performance.propulsion import (
    PropulsionSystem,
    FactorMotor,
    ConstantPropeller,
)
from acdesign.performance.aero import AircraftAero, WingAero, FuseAero
import plotly.express as px

import numpy as np
import pandas as pd
import plotly.graph_objects as go


clarky = UIUCPolar.local("CLARKYB")
rg15 = UIUCPolar.local("RG15C")
sa7038 = UIUCPolar.local("SA7038")
e472 = UIUCPolar.local("E472")

cell_power = 3
propulsion = PropulsionSystem(
    propeller=ConstantPropeller(0.5, 3000),
    motor=FactorMotor(0.65),
)
atm = Atmosphere.alt(0)


In [2]:
def solar_wing(nrows, ncols, section):
    b = nrows * 0.14 + 1
    C = ncols * 0.13 + 0.07
    S = b * C

    return WingAero(b, S, [section], [0, 1])


def solar_plane_aero(nrows, ncols, section):
    wing = solar_wing(nrows, ncols, section)
    fus_length = wing.b / 4
    return AircraftAero(
        wing,
        WingAero(wing.b * 0.2, wing.S * 0.2, [e472], [0, 1]),
        WingAero(wing.b * 0.1, wing.S * 0.1, [e472], [0, 1]),
        FuseAero(fus_length, 0.05),
        0.02,
        fus_length * 0.75,
    )


In [3]:
import xarray as xr


nrows = 30
ncols = 2
mass = np.linspace(4, 20, 10)

wing = solar_wing(nrows, ncols, clarky)


vmd = wing.minimize(lambda row: row.drag, atm, mass * 9.81)


vmp = (
    wing.minimize(
        lambda row: propulsion(atm, row.fs_v, row.drag),
        atm,
        mass * 9.81,
    )
    .rename(columns=dict(minVal="power"))
    .assign(mass=mass, nrows=nrows, ncols=ncols, solar_power=nrows * ncols * cell_power)
)

vmp

,wing_l,fs_v,Cl,Cd0,Cm,k,Cd,lift,drag,moment,power,mass,nrows,ncols,solar_power
5,39.24,6.524158,0.877131,0.018272,-0.068226,0.0202,0.033813,39.24,1.512688,-1.007222,30.366206,4.000000,30,2,180
6,56.68,8.337906,0.775712,0.013874,-0.075402,0.0202,0.026029,56.68,1.901925,-1.818138,48.794074,5.777778,30,2,180
3,74.12,9.378144,0.801837,0.012739,-0.075609,0.0202,0.025726,74.12,2.378082,-2.306406,68.621530,7.555556,30,2,180
4,91.56,10.437433,0.799655,0.012173,-0.075969,0.0202,0.025090,91.56,2.872781,-2.870464,92.259886,9.333333,30,2,180
5,109.00,11.428136,0.794072,0.011610,-0.076399,0.0202,0.024348,109.00,3.342137,-3.460724,117.521223,11.111111,30,2,180
5,126.44,12.354062,0.788223,0.011089,-0.076801,0.0202,0.023640,126.44,3.792057,-4.065530,144.145572,12.888889,30,2,180
4,143.88,13.215211,0.783857,0.010623,-0.077134,0.0202,0.023035,143.88,4.228143,-4.672253,171.925553,14.666667,30,2,180
5,161.32,13.862978,0.798656,0.010501,-0.076635,0.0202,0.023386,161.32,4.723686,-5.108231,201.490344,16.444444,30,2,180
4,178.76,14.621247,0.795584,0.010255,-0.076275,0.0202,0.023041,178.76,5.177106,-5.655596,232.909987,18.222222,30,2,180
4,196.20,15.375705,0.789611,0.009987,-0.075996,0.0202,0.022582,196.20,5.611066,-6.231459,265.458756,20.000000,30,2,180


In [4]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=mass, y=vmd.fs_v, name="min drag speed"))
fig.add_trace(go.Scatter(x=mass, y=vmp.fs_v, name="min power speed"))

fig.add_trace(
    go.Scatter(
        x=mass,
        y=vmp.power,
        name="min power required",
        yaxis="y2",
        line=dict(dash="dash", color="black", width=2),
    )
)
fig.add_trace(
    go.Scatter(
        x=[0, 20],
        y=[cell_power * nrows * ncols, cell_power * nrows * ncols],
        name="available solar power",
        yaxis="y2",
        mode="lines",
        line=dict(dash="dot", color="gray", width=2),
    )
)

fig.update_layout(
    template="simple_white",
    title="Minimum Drag and Power Speeds vs Aircraft Mass, 30 x 2 Panel Wing",
    xaxis=dict(
        title="Mass (kg)",
        range=[4, 20],
    ),
    yaxis_title="Speed (m/s)",
    yaxis2=dict(
        title="Power Required (W)",
        overlaying="y",
        side="right",
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=0.3, xanchor="left", x=0.61),
)

In [5]:
vfactors = np.linspace(0.6, 1.5, 9)
rundf = (
    xr.DataArray(
        np.outer(vmd.fs_v, vfactors),
        dims=["mass", "vfactor"],
        coords={"mass": mass, "vfactor": vfactors},
    )
    .to_dataframe(name="fs_v")
    .reset_index()
    .drop(columns="vfactor")
)

df = pd.concat(
    [rundf.mass, wing(atm, rundf.fs_v, lift=rundf.mass * 9.81, mode="oto")], axis=1
)

df = df.assign(power=propulsion(atm, df.fs_v, df.drag * 1.12))


fig = go.Figure()
for _df in df.groupby("mass"):
    fig.add_trace(
        go.Scatter(
            x=_df[1].fs_v,
            y=_df[1].power,
            mode="lines",
            name=f"Mass: {_df[0]:.1f} kg",
        )
    )

fig.add_hline(
    y=nrows * ncols * cell_power,
    line=dict(color="black", width=2),
    annotation_text="Available Solar Power",
)
fig.update_layout(
    template="simple_white",
    title="Required Power vs Airspeed",
    xaxis_title="Speed (m/s)",
    yaxis_title="Required Power (W)",
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=1, xanchor="left", x=0.05),
)

In [6]:
from functools import partial

mass = np.linspace(1, 20, 20)
drag_multiplier = 1.12

results = []

for altitude in []:#[0, 9000]:
    atm = Atmosphere.alt(altitude)
    for ncols in [1, 2, 3]:
        for nrows in np.arange(10, 40, 4):

            def get_power(row):
                return propulsion(atm, row.fs_v, row.drag * drag_multiplier)

            results.append(
                solar_wing(nrows, ncols, clarky)
                .minimize(
                    get_power,
                    atm,
                    mass * 9.81,
                )
                .assign(
                    altitude=altitude,
                    mass=mass,
                    nrows=nrows,
                    ncols=ncols,
                    solar_power=nrows * ncols * cell_power,
                )
            )

results = pd.concat(results).rename(columns=dict(minVal="power"))

rdf = (
    results.loc[results.power <= results.solar_power]
    .groupby(["altitude", "nrows", "ncols"])
    .apply(lambda df: df.loc[df.mass == df.mass.max()], include_groups=False)
    .reset_index()
)


colors = px.colors.qualitative.Plotly
fig = go.Figure()

for i, ((altitude, ncols), df) in enumerate(rdf.groupby(["altitude", "ncols"])):
    fig.add_trace(
        go.Scatter(
            x=df.nrows,
            y=df.mass,
            mode="lines",
            line=dict(color=colors[i%3], dash="solid" if altitude == 0 else "dash"),
            name=f"{'Sea Level' if altitude==0 else str(altitude) + 'm'}, {ncols} Panel Rows",
        )
    )
fig.update_layout(
    template="simple_white",
    title="Maximum Mass of Solar-Powered Aircraft",
    xaxis_title="Number of Spanwise Solar Cells",
    yaxis_title="Maximum Mass (kg)",
    width=600, height=400,
    legend=dict(yanchor="top", y=1, xanchor="left", x=0.01),
    margin=dict(l=40, r=40, t=40, b=40)
)

ValueError: No objects to concatenate

In [ ]:

from acdesign.avl.parse_avl_output import parse_strip_forces, parse_strip_force_df
from pathlib import Path    
from acdesign.avl.avl_runner import run_avl
from itertools import chain

from acdesign.avl.keywords import kwdict
from acdesign.aircraft.wing import Wing
from acdesign.aircraft.wing_panel import WingPanel

wing = Wing(
    [
        WingPanel.trapezoidal(1.5, 1.5 * 0.4, 1, 0.25),
        #WingPanel.trapz_crct(3, 0.4, 0.2, 0.25),
        WingPanel.elliptical_cr(3, 0.4, 0.25),
    ]
)

avlheader = kwdict["HEADER"](
    "MACE 1", 0, 1, 0, 0, wing.S, wing.smc, wing.b, -wing.smc / 4, 0, 0
)[1:]
avldata = wing.dump_avl(np.linspace(0, 1, 20), sections="flat")
Path("avl/geom.avl").write_text("\n".join(avlheader + avldata))


cls = np.linspace(0.1, 1.0, 5)

def run_cl(name: str, cl: float):
    return [
        f"a c {cl}",
        "x",
        f"ft total_forces_{name}.out",
        f"fs strip_forces_{name}.out",
    ]

print(run_avl(
    [
        "load geom.avl",
        "OPER",
        *chain(*[run_cl(i, cl)  for i, cl in enumerate(cls)]),
        "",
        "QUIT",
    ]
)[1])

cls = np.linspace(0.1, 1.0, 5)

sload_dfs = [parse_strip_force_df(Path(f"avl/strip_forces_{i}.out")) for i in range(len(cls))]
sloads = [parse_strip_forces(Path(f"avl/strip_forces_{i}.out"), wing.b) for i in range(len(cls))]

print(wing)
fig = wing.plot()
fig.update_layout(
    template="simple_white",
    title="Wing Planform",
    xaxis=dict(dtick=0.1),
    yaxis=dict(dtick=0.1),
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=0.01),
    margin=dict(l=40, r=40, t=40, b=40  ),
    width=800, height=300
)


b''
Wing(b=4.50, S=1.54, AR=13.13, smc=0.34, TR=0.00)


In [32]:

fig = wing.plot().update_layout(
    template="simple_white",
    xaxis=dict(dtick=0.1),
    yaxis=dict(dtick=0.1),
    legend=dict(yanchor="top", y=0.7, xanchor="left", x=0.01)
)

y = np.linspace(0, wing.b / 2, 100)

fig2 = go.Figure()
for i, sloaddf in enumerate(sload_dfs):
    fig2.add_trace(
        go.Scatter(
            x=sloaddf.Yle,
            y=sloaddf.c_cl / sloaddf.c_cl.mean(),
            mode="lines",
            name=f"CL={cls[i]:.2f}",
        )
    )
fig2.update_layout(
    template="simple_white",
    title="Lift Distribution",    
    xaxis_title="Spanwise Location (m)",
    yaxis_title="lift_factor",
    width=800, height=300,
    xaxis=dict(dtick=0.1),
    margin=dict(l=40, r=40, t=40, b=40  ),
    legend=dict(visible=False)

)


